In [1]:
import json
import pandas as pd
import numpy as np
import csv
import shutil
import os
import random

In [2]:
#Dataset - https://www.kaggle.com/nih-chest-xrays/data
#create a dataframe for Data_Entry_2017.csv
Data_Entry_CSV_df = pd.read_csv('Data_Entry_2017.csv')

In [5]:
#Create a function to copy images as per our required findings(Pneumonia/Normal) and train/test dataset  
def GetImages(finding_name, train_test):
    # Filter the x-ray images for Pnuemonia patients only.
    finding_df = Data_Entry_CSV_df[Data_Entry_CSV_df['Finding Labels'].eq(finding_name)]
    #finding_df.head()
    
    # finding_df dataframe count, duplicates, remove duplicates
    finding_df.drop_duplicates(subset=['Image Index'])
    finding_df['Image Index'].dropna()
    #finding_df['Image Index'].count() 
    
    # Create a dataframe for test_list.txt
    if train_test == 'test':
        train_test_list_df = pd.read_csv('test_list.txt', header=None)
        train_test_list_df.columns = ['Image Index']
        #train_test_list_df.head()
    else: 
        train_test_list_df = pd.read_csv('train_val_list.txt', header=None)
        train_test_list_df.columns = ['Image Index']
        #train_test_list_df.head()
    
    train_test_list_df.drop_duplicates(subset=['Image Index'])
    #train_test_list_df.count() 
    
    # Merge two dataframes, result will display test images for Pneumonia patients.
    finidngs_train_test_df = pd.merge(finding_df, train_test_list_df, on=["Image Index"]) #inner join
    ##finidngs_train_test_df.head()
    
    # create a text file for image list
    finidngs_train_test_df['Image Index'].to_csv('finidngs_train_test.txt', header=None, index=None)
    
    img_list = finidngs_train_test_df['Image Index'].tolist()
    
    # findings = 'No Findins'(Normal) has test=9500+ and train=50,000+ images, while Pneumonia has test=88 and train = 234 images
    #To keep the balance in both findings,we will get Normal_test = 150 and Normal_train = 400 images.
    if(finding_name == 'No Finding'):
        if(train_test == 'test'):
            img_list = random.sample(img_list, 150)
        else:
            img_list = random.sample(img_list, 400)
        
        
    # Get no of file/copy images
    source_path_list = ["images_001\images\\", "images_002\images\\", "images_003\images\\", "images_004\images\\", "images_005\images\\",
                    "images_006\images\\", "images_007\images\\", "images_008\images\\", "images_009\images\\", "images_010\images\\",
                    "images_011\images\\", "images_012\images\\"]

    destination = "data_clean\\"

    i = 0 
    #file_list = os.listdir(source_path)
    #print(file_list)
    for source_path in source_path_list:
        for img_name in img_list:
            #print(source_path+img_name)
            #i = i+1
            if os.path.exists(source_path+img_name):
                shutil.copy(source_path+img_name, destination+img_name)
                #print(f'Image ' + img_name + ' exist in ' + source_path)
                i = i+1
                
    print(f'Total '+ str(i) + ' images are copied into data_clean folder.')
    print(f'These images are for ' + finding_name +' findings and ' + train_test + ' dataset.')
    

In [6]:
# call GetImages(finding_name, train_test)
GetImages('No Finding', 'train')

Total 400 images are copied into data_clean folder.
These images are for No Finding findings and train dataset.
